In [3]:
import sys
sys.path.append('/MS3D')
from pcdet.utils import box_fusion_utils
from tqdm import tqdm 
import numpy as np

dets_txt = '/MS3D/tools/cfgs/target_waymo/kbf_combinations/pv_vox_c.txt'

ps_dict = {}
det_annos = box_fusion_utils.load_src_paths_txt(dets_txt)
print('Number of detectors: ', len(det_annos))
combined_dets = box_fusion_utils.combine_box_pkls(det_annos, score_th=0.1)

Number of detectors:  24


aggregating_proposals: 100%|██████████████| 18840/18840 [05:20<00:00, 58.70it/s]


In [18]:
mydict = {}
for frame_idx, frame_dets in enumerate(combined_dets):
    mydict[frame_idx] = {}
    mydict[frame_idx]['ped_count'] = np.count_nonzero(frame_dets['names'] == 'Pedestrian')
    mydict[frame_idx]['cyc_count'] = np.count_nonzero(frame_dets['names'] == 'Cyclist')

In [26]:
keep_idx = []
for k,v in mydict.items():
    if (v['ped_count'] < 100) or (v['cyc_count'] < 100):
        continue
    keep_idx.append(k)
print(len(keep_idx))

2205


In [27]:
sampled_cdets = []
for idx in keep_idx:
    sampled_cdets.append(combined_dets[idx])

In [30]:
from pcdet.utils import generate_ps_utils 
generate_ps_utils.save_data(sampled_cdets, '/MS3D/tools/cfgs/target_waymo/kbf_combinations', 
                            name=f"pv_vox_c_combined_dets_2205_lotsofpedcyc.pkl")